# Data Madness
## Group idk

### authors:
* Simon Garland
* PierrePaul Charbonnier
* Filip Straka

Department of Advanced Computing Sciences
Maastricht University
March 2024

Let's begin with extracting the data from the ZIP files (they'll grow tremendously)

In [25]:
import os.path

# NB: im doing it on a small fraction of the data now to prevent my laptop from blowing up ever so nicely

## imports ##

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os

In [26]:
# define all possible files
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
years = ["21", "22", "23", "24"]
modes = ["standard", "rapid", "blitz"]

In [27]:
for month in months:
    for mode in modes:
        #for year in years:
        year = "24"
        path = '../data_zipped/'+mode+"_"+month+year+"frl_xml.zip"
        if os.path.exists(path):
            with zipfile.ZipFile(path, 'r') as zip_ref:
                zip_ref.extractall('../data_unzipped')

In [ ]:
lst = []
for month in months:
    for mode in modes:
        year = "24"
        #for year in years:
        path = '../data_unzipped/'+mode+"_"+month+year+"frl_xml.xml"
        if os.path.exists('../data_unzipped/'+mode+"_"+month+year+"frl_xml.xml"):
            df = pd.read_xml(path)
            df.head(10)
            lst.append(df)